# Getting things going and highlighting "the problem"

In [1]:
import pandas as pd
import sqlite3

ramphs = pd.read_csv("https://raw.githubusercontent.com/sfsheath/roman-amphitheaters/master/roman-amphitheaters.csv")
chronogrps = pd.read_csv("https://raw.githubusercontent.com/sfsheath/roman-amphitheaters/master/chronogrps.csv")

# I put an uncompressed version of pleiades-places into my folder on the github repo
pleiades = pd.read_csv("https://raw.githubusercontent.com/isaw-ga-3024/isaw-ga-3024.github.io/master/heath-sebastian/pleiades-places.csv")

# One note: because it takes a long time to load the pleiades data,
# avoid running this cell again unless necessary
# As I look back on class, perhaps this is why Mikael reported 
# that his replace was taking a long time. Was the data reloading across the internet?

In [2]:
# ramphs, chronogrps, and pleiades are now pandas DataFrames

print(type(ramphs))
print(type(chronogrps))
print(type(pleiades))

# Again, these are pandas DataFrames, not sql tables!

<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>


In [8]:
# OK, to our problem in class.

# inspect the values in both DataFrmes
print(ramphs['pleiades'].head(2).str.replace("http://pleiades.stoa.org/places/", ""))
print(pleiades['id'].head(2))

# They're different. Bummer. 

0    893989
1    148217
Name: pleiades, dtype: object
0    265876
1    265877
Name: id, dtype: int64


# A tiny bit of string manipulation

In [5]:
tmp = "Hello World"
tmp.replace("Hello", "") # note that this doesn't remove the space

' World'

# A little bit of pandas dataframe manipulation

In [10]:
# first
ramphs.head(2)

,id,title,label,pleiades,type,capacity,modcountry,chronogrp,certainty,youtube,extmajor,extminor,arenamajor,arenaminor,latitude,longitude,elevation
0,duraEuroposAmphitheater,Amphitheater at Dura Europos,Dura,http://pleiades.stoa.org/places/893989,amphitheater,1000,Syria,Severan,NaN,NaN,50,44,31,25,34.749855,40.728926,223
1,arlesAmphitheater,Amphitheater at Arles,Arles,http://pleiades.stoa.org/places/148217,amphitheater,20000,France,Flavian,NaN,https://www.youtube.com/watch?v=oCz-76hb1LU,136,107,47,32,43.677778,4.631111,21


In [11]:
# then
ramphs['pleiades'].head(2)

# so it's easy to output either all columns, or just a selection.
# And it's likely that your notebook is trying to nicely format the output of ramphs.head(2).

0    http://pleiades.stoa.org/places/893989
1    http://pleiades.stoa.org/places/148217
Name: pleiades, dtype: object

In [12]:
# it's easy to create new columns
ramphs['new'] = "test"
ramphs['new'].head(5)

# And look!!!! Very important to note that pandas will automatically assign "test" to all rows.
# That's one of its great powers.

0    test
1    test
2    test
3    test
4    test
Name: new, dtype: object

In [13]:
# combining pandas and string manipulation
ramphs['new'].str.replace("e","").head(5) # do note the ".str" before ".replace"

# Again, pandas calculates result for all rows

0    tst
1    tst
2    tst
3    tst
4    tst
Name: new, dtype: object

In [14]:
# here's a useful construct, though dangerous in that 
# it changes data (a concern some of you raised in class)


ramphs['new'] = ramphs['new'].str.replace("t","")
ramphs['new'].head(5)

0    es
1    es
2    es
3    es
4    es
Name: new, dtype: object

# Review of adding tables to an sql database (to keep our terminology straight)

In [15]:
# The conn variable is a 'database connection'
# we are making this database in memory, meaning it won't be saved
conn = sqlite3.connect(':memory:')

# as convenience, I've put the above in a separate cell. You may not need to run it again.


In [18]:
# the following two lines create sql tables from pandas DataFrames
ramphs.to_sql('ramphs',conn,if_exists="replace")
chronogrps.to_sql('chronogrps', conn, if_exists="replace")

# copy-paste, then edit one of the the above lines to make a pleiades table
pleiades.to_sql('pleiades', conn, if_exists="replace")


In [19]:
# let's test that we have sql tables
pd.read_sql("""SELECT id,pleiades FROM RAMPHS WHERE capacity > 40000""",conn)

# but that only confirms all is well working for the ramphs table.
# Good to test others as well.

,id,pleiades
0,romeFlavianAmphitheater,http://pleiades.stoa.org/places/423025


In [20]:
# a reminder
pd.read_sql("""PRAGMA table_info(pleiades)""",conn)
# will display the columns in the pleiades table (assuming you called it 'pleiades')
# look at those columns, which ones have 'chronological' information

,cid,name,type,notnull,dflt_value,pk
0,0,index,INTEGER,0,None,0
1,1,authors,TEXT,0,None,0
2,2,bbox,TEXT,0,None,0
3,3,connectsWith,TEXT,0,None,0
4,4,created,TEXT,0,None,0
5,5,creators,TEXT,0,None,0
6,6,currentVersion,INTEGER,0,None,0
7,7,description,TEXT,0,None,0
8,8,extent,TEXT,0,None,0
9,9,featureTypes,TEXT,0,None,0


# More mixing of pandas and sql

In [22]:
# worth noting that the result of pd.read_sql statement is a pandas DataFrame
type(pd.read_sql("""SELECT id FROM ramphs WHERE capacity > 20000""",conn))

# this means you can use pandas .groupby on it. This is familiar from last week
pd.read_sql("""SELECT modcountry,capacity FROM ramphs""",conn).groupby('modcountry').count()


,capacity
modcountry,
Albania,1
Algeria,6
Austria,3
Bulgaria,2
Croatia,2
Cyprus,0
France,18
Germany,2
Greece,1


In [24]:
# You now have all the piecces to move towards completing and going beyond
# the task I set in class. The task was:

# "For each amphitheater with a Pleiades ID,
# list that ID's ancient name(s) and chronological information
# First step... what are the steps?"

# At a minimum, write an SQL SELECT query that lists amphitheater IDs that have
# Pleiades identifiers along with the chronological information that pleiades records

# MORE INTERESTING AND HARDER: Write an SQL SELECT query that also lists the start column
# from the chronogrps table.
# So four columns at least: ramphs.id, chronogrps.start, pleiades.minDate, pleiades.maxDate

# once you do this it's easy to imagine a visualization of the difference between the founding
# of a city as indicated by pleiades and the construction of its amphitheater.
# A histogram of this would be nice. I'd calculate the difference using panda.
# (Or rather, that's how I did it.)

# Or think of some other manipulation that goes a step beyond the "at a minumum" query.


# Finally, as you ponder how to do the above, think about the conceptual range of DH as 
# found in this week's readings and as experienced while manipulating data in this
# ipython notebook. It's all DH and we love it all. How can theory and practice fit together?
# What other readings we've been assigned are relevant. This is just fodder for conversation.

In [27]:
# "For each amphitheater with a Pleiades ID,
# list that ID's ancient name(s) and chronological information
# First step... what are the steps?"


ramphs['pleiades'] = ramphs['pleiades'].str.replace("http://pleiades.stoa.org/places/","")
ramphs.to_sql('ramphs',conn,if_exists="replace")
pd.read_sql("""SELECT ramphs.pleiades, pleiades.id FROM ramphs, pleiades LIMIT 10""", conn)

,pleiades,id
0,893989,265876
1,893989,265877
2,893989,265878
3,893989,265880
4,893989,265882
5,893989,265883
6,893989,265884
7,893989,265886
8,893989,265887
9,893989,265888


In [31]:
# MORE INTERESTING AND HARDER: Write an SQL SELECT query that also lists the start column
# from the chronogrps table.
# So four columns at least: ramphs.id, chronogrps.start, pleiades.minDate, pleiades.maxDate

pd.read_sql("""SELECT ramphs.id, ramphs.title, ramphs.chronogrp, chronogrps.start, pleiades.minDate, pleiades.maxDate 
                FROM ramphs, chronogrps, pleiades 
                WHERE (ramphs.pleiades=pleiades.id) AND (ramphs.chronogrp=chronogrps.chronogrp) 
                ORDER BY (chronogrps.start)""",conn)

,id,title,chronogrp,start,minDate,maxDate
0,pompeiiAmphitheater,Amphitheater at Pompeii,Republican,-70,-750,2100
1,cumaeAmphitheater,Amphitheater at Cumae,Republican,-70,-750,640
2,pozzuoliEarlyAmphitheater,Early Amphitheater at Pozzuoli,Republican,-70,-750,2100
3,paestumAmphitheater,Amphitheater at Paestum,Republican,-70,-750,2100
4,avellaAmphitheater,Amphitheater at Abella,Republican,-70,-330,640
5,ferentoAmphitheater,Amphitheater at Ferento,Republican,-70,-550,640
6,sutriumAmphitheatre,Amphitheater at Sutrium,Republican,-70,-550,640
7,telesiaAmphitheatre,Amphitheater at Telesia,Republican,-70,-330,640
8,antiochAmphitheater,Amphitheater at Antioch,Republican,-70,-1750,2100
9,beneventoAmphitheater,Amphitheater at Benevento,Republican,-70,-330,2100


In [35]:
#with the sites listed and start and end dates included

pd.read_sql("""
SELECT ramphs.title, ramphs.chronogrp, chronogrps.start, chronogrps.end, pleiades.title, pleiades.minDate, pleiades.maxDate
FROM ramphs, pleiades, chronogrps
WHERE (ramphs.pleiades = pleiades.id) AND (ramphs.chronogrp = chronogrps.chronogrp)
ORDER BY chronogrps.start
""", conn)

,title,chronogrp,start,end,title,minDate,maxDate
0,Amphitheater at Pompeii,Republican,-70,-31,Pompeii,-750,2100
1,Amphitheater at Cumae,Republican,-70,-31,Cumae/Kyme (Campanian),-750,640
2,Early Amphitheater at Pozzuoli,Republican,-70,-31,Dikaiarcheia/Puteoli,-750,2100
3,Amphitheater at Paestum,Republican,-70,-31,Poseidonia/Paestum,-750,2100
4,Amphitheater at Abella,Republican,-70,-31,Abella,-330,640
5,Amphitheater at Ferento,Republican,-70,-31,Lucus Feroniae,-550,640
6,Amphitheater at Sutrium,Republican,-70,-31,Sutrium,-550,640
7,Amphitheater at Telesia,Republican,-70,-31,Telesia,-330,640
8,Amphitheater at Antioch,Republican,-70,-31,Antiochia/Theoupolis,-1750,2100
9,Amphitheater at Benevento,Republican,-70,-31,Beneventum/Maleventum,-330,2100


# What if I don't accept the assignment?

In [ ]:
# That's OK. You can always use your own data or data you download. But you need to turn in
# an iPython notebook that demonstrates skills of approximately the same level of difficulty
# as what is above. At a minimum: a many-to-one or one-to-many relationship that is queried
# via SQL and then used to say something minimally interesting. The usual "go for it" applies.